<a href="https://colab.research.google.com/github/MaximoDouglas/deep-learning-with-python-brownlee/blob/master/code/colab/part_three/mlp_diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get file

In [0]:
!wget "http://ftp.ics.uci.edu/pub/machine-learning-databases/pima-indians-diabetes/pima-indians-diabetes.data" -O pima-indians-diabetes.csv

# Code - First example direct from the book

In [0]:
from keras.models import Sequential
from keras.layers import Dense
import numpy

# fix random seed for reproducibility - it allows that no matter if we execute 
# the code more than one time, the random values have to be the same
seed = 7
numpy.random.seed(seed)

# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")

# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

# create model
model = Sequential()
model.add(Dense(12, input_dim=8, activation="relu", kernel_initializer="uniform"))
model.add(Dense(8, activation="relu", kernel_initializer="uniform"))
model.add(Dense(1, activation="sigmoid", kernel_initializer="uniform"))

# Compile model 
# binary_crossentropy = logarithmic loss
# adam = gradient descent algorithm
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(X, Y, epochs=150, batch_size=10, verbose=0)

# Evaluating model with the training data
scores = model.evaluate(X, Y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# Code - Manual k-Fold Cross Validation (adapted)


In [0]:
# MLP for Pima Indians Dataset with 10-fold cross validation
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import StratifiedKFold
import numpy

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")

# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []

for (train, test) in kfold.split(X, Y):
  # create model
  model = Sequential()
  model.add(Dense(12, input_dim=8, activation="relu", kernel_initializer="uniform"))
  model.add(Dense(8, activation="relu", kernel_initializer="uniform"))
  model.add(Dense(1, activation="sigmoid", kernel_initializer="uniform"))
  
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  # Fit the model
  history = model.fit(X[train], Y[train], epochs=150, batch_size=10, verbose=0)
  
  # evaluate the model
  scores = model.evaluate(X[test], Y[test], verbose=0)
  print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
  cvscores.append(scores[1] * 100)
  
print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))

# Code - Keras and Scikit-learn (adapted)

In [0]:
# MLP for Pima Indians Dataset with 10-fold cross validation via sklearn
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import numpy

# Function to create model, required for KerasClassifier
def create_model():
  # create model
  model = Sequential()
  model.add(Dense(12, input_dim=8, activation="relu", kernel_initializer="uniform"))
  model.add(Dense(8, activation="relu", kernel_initializer="uniform"))
  model.add(Dense(1, activation="sigmoid", kernel_initializer="uniform"))
  
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")

# split into input (X) and output (Y) variables
X = dataset[:,0:8]
y = dataset[:,8]

# create model
model = KerasClassifier(build_fn=create_model, epochs=150, batch_size=10)

# evaluate using 10-fold cross validation
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(model, X, y, cv=kfold)
print("Acc: %.2f%%"%((results.mean())*100))

# Code - Grid Search (adapted)

In [0]:
# MLP for Pima Indians Dataset with grid search via sklearn
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import numpy

# Function to create model, required for KerasClassifier
def create_model(optimizer="rmsprop", init="glorot_uniform"):
  # create model
  model = Sequential()
  model.add(Dense(12, input_dim=8, activation="relu", kernel_initializer=init))
  model.add(Dense(8, activation="relu", kernel_initializer=init))
  model.add(Dense(1, activation="sigmoid", kernel_initializer=init))

  # Compile model
  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  return model

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")

# split into input (X) and output (Y) variables
X = dataset[:,0:8]
y = dataset[:,8]

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# grid search epochs, batch size and optimizer
optimizers = ['rmsprop', 'adam']
init = ['glorot_uniform', 'normal', 'uniform']
eps = numpy.array([50, 100, 150])
batches = numpy.array([5, 10, 20])

param_grid = dict(optimizer=optimizers, epochs=eps, batch_size=batches, init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X, y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

#The way the book print the summarization of other params combinations is 
# deprecate. So, in my code I had to adapt the new grid_result return to print
# like in the book. The altered code is in 
# "Code - Grid Search (alterede by Maximo, D.H.)" jupyter section, or, in 
# github: /deep_learning/code/local_env/009_mlp_diabetes_GSearch-maximo.py